In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.metrics import r2_score
from PIL import Image
import re

warnings.filterwarnings('ignore')
import catboost as cb
from catboost import CatBoostRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Using CLIP to create image-text embeddings, then classify each image to provided text category

In [ ]:
from transformers import CLIPProcessor, CLIPModel

clipModel = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
#stores images and ids
image_list = []
image_homeids = []

In [ ]:
#test categories to be used with CLIP model, for house quality scoring

# sizes = ["Small house", "Medium house", "Large house"]
styles = ["Newer house", "Older house"]
conditions = [
    "Well-maintained house",
    "House in need of minor repairs",
    "House in need of major renovations"
]

categories = [
    # f"{size}, {style}, {condition}"
    # for size in sizes
    f"{style}, {condition}"
    for style in styles
    for condition in conditions
]

for i, combination in enumerate(categories, 1):
    print(f"{i}. {combination}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clipModel.to(device)
text_inputs = processor(text=categories, return_tensors="pt", padding=True)
text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

1. Newer house, Well-maintained house
2. Newer house, House in need of minor repairs
3. Newer house, House in need of major renovations
4. Older house, Well-maintained house
5. Older house, House in need of minor repairs
6. Older house, House in need of major renovations


In [ ]:
count = 0
for anImg in os.listdir('/content/drive/MyDrive/!mlData/info558Kaggle/SimPictures'):
  imgHomeId = anImg[5:10]
  #remove trailing 0s
  imgHomeId = imgHomeId.lstrip('0')
  if imgHomeId not in image_homeids:
    image_list.append(Image.open(f'/content/drive/MyDrive/!mlData/info558Kaggle/SimPictures/{anImg}').convert("RGB"))
    image_homeids.append(imgHomeId)
    count += 1
print(count)


1200


In [ ]:
#run clip model on text-image inputs to get text-image embeddings
inputs = processor(text=categories, images=image_list, return_tensors="pt", padding=True)
inputs = {key: value.to(device) for key, value in inputs.items()}
with torch.no_grad():
    outputs = clipModel(**inputs)
#get logies
logits_per_image = outputs.logits_per_image
#see category mapping
predicted_indices = torch.argmax(logits_per_image, dim=1)
predicted_categories = [categories[idx] for idx in predicted_indices]

In [ ]:
clip_results_df = pd.DataFrame({
    "homeid": [int(homeid) for homeid in image_homeids],
    "predicted_category": predicted_categories
})
clip_results_df.head()

,homeid,predicted_category
0,5734,"Newer house, House in need of major renovations"
1,4744,"Newer house, House in need of major renovations"
2,7704,"Older house, House in need of minor repairs"
3,6774,"Newer house, House in need of minor repairs"
4,1626,"Newer house, House in need of major renovations"


House reparis estimation text file classificaiton with zero-short transformers

In [ ]:
from transformers import pipeline
bartClassifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=device)

In [ ]:
repair_degree = ['low repair costs', 'medium repair costs', 'high repair costs']

In [ ]:
txtCount = 0
extracted_txt = []
txt_homeids = []
for aTxt in os.listdir('/content/drive/MyDrive/!mlData/info558Kaggle/SimReports'):
  aTxt_homeid = aTxt[5:10]
  aTxt_homeid = aTxt_homeid.lstrip('0')
  txtContent = open(f'/content/drive/MyDrive/!mlData/info558Kaggle/SimReports/{aTxt}', 'r').read()

  extracted_txt.append(txtContent)
  txt_homeids.append(aTxt_homeid)
  txtCount += 1
print(txtCount)

1200


In [ ]:
bartResults = bartClassifier(extracted_txt, candidate_labels=repair_degree)

In [ ]:
curCount =0
for text, result in zip(extracted_txt, bartResults):
    curCount +=1
    print(f"Text: {text}")
    print(f"Predicted category: {result['labels'][0]} with score: {result['scores'][0]:.2f}")
    print()
    if curCount == 5:
      break

Text: Inspection Report of Home ID # 05335:

This single-family property, located in Lothal, was built in 2024, featuring 6 bedrooms and 5.5 bathrooms, across a total living space of 1933 square feet. It is a 2-story construction and has been sold 1 time.

Key Findings from Inspection:

Upon entering the home, it is immediately apparent that significant neglect has led to various signs of deterioration and damage throughout the property. The interior paint has extensively peeled, and there are visible stains on many walls, suggesting water damage or mold issues. Flooring throughout the house, particularly in high-traffic areas, shows severe wear with numerous scratched, warped, or cracked sections. The kitchen and bathrooms exhibit outdated fixtures and considerable wear; cabinets and countertops are chipped and stained, and several plumbing fixtures are in disrepair, showing signs of leaks and rust. Electrical outlets and fixtures are outdated and not up to current safety standards. T

In [ ]:
classified_repair_results = [result["labels"][0] for result in bartResults]
repair_results_df = pd.DataFrame({
    "homeid": [int(homeid) for homeid in txt_homeids],
    "predicted_category": classified_repair_results
})
repair_results_df.head()

,homeid,predicted_category
0,5335,high repair costs
1,6207,high repair costs
2,8393,high repair costs
3,5976,high repair costs
4,4552,high repair costs


Merging all generated feature dfs back to orig df

In [ ]:
#load in original data first
data = pd.read_csv('/content/drive/MyDrive/!mlData/info558Kaggle/df-train_1000x12.csv')
test_data = pd.read_csv('/content/drive/MyDrive/!mlData/info558Kaggle/df-test_200x11.csv')
print(data.dtypes)
print(data.isnull().sum())
print(len(data))

homeid                int64
feat_yrbuilt          int64
feat_nrooms           int64
feat_nbed             int64
feat_nbath          float64
feat_sqft             int64
feat_proptype        object
feat_basement        object
feat_stories         object
feat_solar           object
feat_nsales           int64
target_homevalue      int64
dtype: object
homeid              0
feat_yrbuilt        0
feat_nrooms         0
feat_nbed           0
feat_nbath          0
feat_sqft           0
feat_proptype       0
feat_basement       0
feat_stories        0
feat_solar          0
feat_nsales         0
target_homevalue    0
dtype: int64
1000


In [ ]:
data = data.merge(clip_results_df, on="homeid", how="left")
data = data.merge(repair_results_df, on="homeid", how="left")
print(data.dtypes)
data.head()

homeid                    int64
feat_yrbuilt              int64
feat_nrooms               int64
feat_nbed                 int64
feat_nbath              float64
feat_sqft                 int64
feat_proptype            object
feat_basement            object
feat_stories             object
feat_solar               object
feat_nsales               int64
target_homevalue          int64
predicted_category_x     object
predicted_category_y     object
dtype: object


,homeid,feat_yrbuilt,feat_nrooms,feat_nbed,feat_nbath,feat_sqft,feat_proptype,feat_basement,feat_stories,feat_solar,feat_nsales,target_homevalue,predicted_category_x,predicted_category_y
0,5936,2024,16,2,7.0,3456,Condominium,no,single,no,2,151512,"Newer house, House in need of minor repairs",high repair costs
1,7314,1899,13,2,1.0,500,Condominium,no,single,no,1,356344,"Newer house, House in need of minor repairs",high repair costs
2,5448,2024,7,2,9.0,7414,SingleFam,no,single,no,2,78112,"Newer house, House in need of major renovations",high repair costs
3,8560,2007,11,2,5.5,6619,SingleFam,no,single,no,3,872768,"Newer house, House in need of major renovations",high repair costs
4,5783,2024,15,3,4.5,3226,MultiFam,no,multiple,no,1,946086,"Newer house, House in need of minor repairs",high repair costs


In [ ]:
test_data = test_data.merge(clip_results_df, on="homeid", how="left")
test_data = test_data.merge(repair_results_df, on="homeid", how="left")
print(test_data.dtypes)
test_data.head()

homeid                    int64
feat_yrbuilt              int64
feat_nrooms               int64
feat_nbed                 int64
feat_nbath              float64
feat_sqft                 int64
feat_proptype            object
feat_basement            object
feat_stories             object
feat_solar               object
feat_nsales               int64
predicted_category_x     object
predicted_category_y     object
dtype: object


,homeid,feat_yrbuilt,feat_nrooms,feat_nbed,feat_nbath,feat_sqft,feat_proptype,feat_basement,feat_stories,feat_solar,feat_nsales,predicted_category_x,predicted_category_y
0,4997,1734,8,7,6.5,4500,MultiFam,yes,multiple,no,2,"Newer house, House in need of major renovations",high repair costs
1,7647,2024,10,3,4.0,4470,Condominium,no,single,no,0,"Newer house, House in need of major renovations",high repair costs
2,8202,1692,13,2,3.0,3985,SingleFam,no,single,no,2,"Newer house, House in need of minor repairs",high repair costs
3,5573,2024,8,2,6.0,2079,SingleFam,no,single,no,3,"Newer house, House in need of major renovations",high repair costs
4,4612,2024,3,2,7.5,500,MultiFam,yes,multiple,no,2,"Newer house, House in need of minor repairs",high repair costs


In [ ]:
#merge data with clip_results_df
dissimilar_rows = data[data["predicted_category_x"] != data["predicted_category_y"]]
if not dissimilar_rows.empty:
    print("Dissimilar rows found:")
    print(dissimilar_rows)
else:
    data = data.drop("predicted_category_y", axis=1)
    data = data.rename(columns={"predicted_category_x": "predicted_category"})
    print("No dissimilar rows found.")
print(data.dtypes)
data.head()

Dissimilar rows found:
     homeid  feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  \
0      5936          2024           16          2         7.0       3456   
1      7314          1899           13          2         1.0        500   
2      5448          2024            7          2         9.0       7414   
3      8560          2007           11          2         5.5       6619   
4      5783          2024           15          3         4.5       3226   
..      ...           ...          ...        ...         ...        ...   
995     466          1931           15          4         4.5       6187   
996    6367          1852            3          6         7.5       3779   
997    9678          1357            8          2         1.0        500   
998    6403          1397            3          2         1.0       6168   
999    9403          2024            4          3         1.0       5981   

    feat_proptype feat_basement feat_stories feat_solar  feat_ns

,homeid,feat_yrbuilt,feat_nrooms,feat_nbed,feat_nbath,feat_sqft,feat_proptype,feat_basement,feat_stories,feat_solar,feat_nsales,target_homevalue,predicted_category_x,predicted_category_y
0,5936,2024,16,2,7.0,3456,Condominium,no,single,no,2,151512,"Newer house, House in need of minor repairs",high repair costs
1,7314,1899,13,2,1.0,500,Condominium,no,single,no,1,356344,"Newer house, House in need of minor repairs",high repair costs
2,5448,2024,7,2,9.0,7414,SingleFam,no,single,no,2,78112,"Newer house, House in need of major renovations",high repair costs
3,8560,2007,11,2,5.5,6619,SingleFam,no,single,no,3,872768,"Newer house, House in need of major renovations",high repair costs
4,5783,2024,15,3,4.5,3226,MultiFam,no,multiple,no,1,946086,"Newer house, House in need of minor repairs",high repair costs


In [ ]:
def encode_cols(input_data):
  encoder = LabelEncoder()
  for col in input_data.columns:
      if input_data[col].dtype == 'object':
          input_data[col] = encoder.fit_transform(input_data[col])
  print(input_data.head())
  return input_data

In [ ]:
data = encode_cols(data)

   homeid  feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  \
0    5936          2024           16          2         7.0       3456   
1    7314          1899           13          2         1.0        500   
2    5448          2024            7          2         9.0       7414   
3    8560          2007           11          2         5.5       6619   
4    5783          2024           15          3         4.5       3226   

   feat_proptype  feat_basement  feat_stories  feat_solar  feat_nsales  \
0              0              0             1           0            2   
1              0              0             1           0            1   
2              2              0             1           0            2   
3              2              0             1           0            3   
4              1              0             0           0            1   

   target_homevalue  predicted_category_x  predicted_category_y  
0            151512                     1   

In [ ]:
X=data.drop(['target_homevalue','homeid'],axis=1)
y=data['target_homevalue']
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=412)

train_dataset = cb.Pool(X_train, y_train)
test_dataset = cb.Pool(X_test, y_test)

   feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  feat_proptype  \
0          2024           16          2         7.0       3456              0   
1          1899           13          2         1.0        500              0   
2          2024            7          2         9.0       7414              2   
3          2007           11          2         5.5       6619              2   
4          2024           15          3         4.5       3226              1   

   feat_basement  feat_stories  feat_solar  feat_nsales  predicted_category_x  \
0              0             1           0            2                     1   
1              0             1           0            1                     1   
2              0             1           0            2                     0   
3              0             1           0            3                     0   
4              0             0           0            1                     1   

   predicted_category_y  


Model training-CatBoost

In [ ]:
catBoostModel = cb.CatBoostRegressor(loss_function="RMSE", task_type="GPU")

In [ ]:
grid = {'iterations': [25, 50, 100, 150],
        'learning_rate': [0.001, 0.03, 0.1],
        'depth': [2, 4, 6, 8, 10],
        'l2_leaf_reg': [0.2, 0.5, 1, 3, 10, 15]}
catBoostModel.grid_search(grid, train_dataset)

0:	learn: 631043.6868046	test: 615885.3937648	best: 615885.3937648 (0)	total: 6.47ms	remaining: 155ms
1:	learn: 630596.2317934	test: 615416.5457119	best: 615416.5457119 (1)	total: 11.8ms	remaining: 135ms
2:	learn: 630161.9790244	test: 614951.0275708	best: 614951.0275708 (2)	total: 16.7ms	remaining: 122ms
3:	learn: 629742.3711354	test: 614504.0125291	best: 614504.0125291 (3)	total: 21.6ms	remaining: 114ms
4:	learn: 629326.1699208	test: 614076.2880555	best: 614076.2880555 (4)	total: 27ms	remaining: 108ms
5:	learn: 628911.1521504	test: 613637.9273806	best: 613637.9273806 (5)	total: 32ms	remaining: 101ms
6:	learn: 628478.8633798	test: 613166.4416649	best: 613166.4416649 (6)	total: 36.9ms	remaining: 94.9ms
7:	learn: 628035.6542439	test: 612713.4826028	best: 612713.4826028 (7)	total: 42ms	remaining: 89.2ms
8:	learn: 627613.2255076	test: 612261.4514993	best: 612261.4514993 (8)	total: 46.8ms	remaining: 83.3ms
9:	learn: 627166.1025942	test: 611814.9136280	best: 611814.9136280 (9)	total: 51.8ms	

Streaming output truncated to the last 5000 lines.
33:	learn: 377093.7873283	test: 374232.8450107	best: 374232.8450107 (33)	total: 387ms	remaining: 182ms
34:	learn: 373083.5478592	test: 370825.2033376	best: 370825.2033376 (34)	total: 394ms	remaining: 169ms
35:	learn: 369271.0687308	test: 367123.7796624	best: 367123.7796624 (35)	total: 401ms	remaining: 156ms
36:	learn: 365536.9846787	test: 363303.5254043	best: 363303.5254043 (36)	total: 409ms	remaining: 144ms
37:	learn: 361971.3446161	test: 360506.7952048	best: 360506.7952048 (37)	total: 415ms	remaining: 131ms
38:	learn: 358626.5980811	test: 357981.0854623	best: 357981.0854623 (38)	total: 423ms	remaining: 119ms
39:	learn: 355253.5899033	test: 355215.9546687	best: 355215.9546687 (39)	total: 445ms	remaining: 111ms
40:	learn: 351678.5592691	test: 352379.1040706	best: 352379.1040706 (40)	total: 454ms	remaining: 99.7ms
41:	learn: 348712.9350420	test: 349344.3760738	best: 349344.3760738 (41)	total: 460ms	remaining: 87.7ms
42:	learn: 345650.38

Training on fold [0/3]
0:	learn: 573900.9029545	test: 601128.2356484	best: 601128.2356484 (0)	total: 9.53ms	remaining: 943ms
1:	learn: 534076.7248189	test: 560790.9265630	best: 560790.9265630 (1)	total: 18.1ms	remaining: 885ms
2:	learn: 500248.4986425	test: 528787.6424745	best: 528787.6424745 (2)	total: 24.6ms	remaining: 796ms
3:	learn: 469385.7982770	test: 495539.7020542	best: 495539.7020542 (3)	total: 31.1ms	remaining: 745ms
4:	learn: 441118.3081864	test: 472143.7131427	best: 472143.7131427 (4)	total: 39.3ms	remaining: 746ms
5:	learn: 416331.3495030	test: 447287.3878006	best: 447287.3878006 (5)	total: 47.4ms	remaining: 743ms
6:	learn: 398096.5014443	test: 427452.1240966	best: 427452.1240966 (6)	total: 53.3ms	remaining: 708ms
7:	learn: 379488.6771011	test: 410573.3501597	best: 410573.3501597 (7)	total: 59.7ms	remaining: 686ms
8:	learn: 363925.1719135	test: 394856.0255829	best: 394856.0255829 (8)	total: 66ms	remaining: 668ms
9:	learn: 348110.5425196	test: 380006.8491973	best: 380006.84

{'params': {'depth': 6,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.2,
  'iterations': 100},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               4

In [ ]:
pred = catBoostModel.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

Testing performance
RMSE: 315783.11
R2: 0.51


In [ ]:
def submit_csv(test_data, model):
  test_data_input = test_data.drop(['homeid'], axis=1)
  test_data_input = encode_cols(test_data_input)
  test_pred = model.predict(test_data_input)
  submission = pd.DataFrame({'homeid': test_data['homeid'], 'target_homevalue': test_pred})
  submit_file_path = '/content/drive/MyDrive/!mlData/info558Kaggle/submission.csv'
  submission.to_csv(submit_file_path, index=False)

In [ ]:
submit_csv(test_data, catBoostModel)

   feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  feat_proptype  \
0          1734            8          7         6.5       4500              1   
1          2024           10          3         4.0       4470              0   
2          1692           13          2         3.0       3985              2   
3          2024            8          2         6.0       2079              2   
4          2024            3          2         7.5        500              1   

   feat_basement  feat_stories  feat_solar  feat_nsales  predicted_category_x  \
0              1             0           0            2                     0   
1              0             1           0            0                     0   
2              0             1           0            2                     1   
3              0             1           0            3                     0   
4              1             0           0            2                     1   

   predicted_category_y  


Linear Regression performance

In [ ]:
def linear_reg_results(X_train, y_train, X_test, y_test):
  from sklearn.linear_model import LinearRegression
  from sklearn.metrics import mean_squared_error, r2_score
  reg = LinearRegression().fit(X_train, y_train)
  score = reg.score(X_train, y_train)
  y_pred = reg.predict(X_test)
  rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
  r2 = r2_score(y_test, y_pred)
  print("Testing performance")
  print("RMSE: {:.2f}".format(rmse))
  print("R2: {:.2f}".format(r2))

In [ ]:
linear_reg_results(X_train, y_train, X_test, y_test)

Testing performance
RMSE: 308257.54
R2: 0.53
